<a href="https://colab.research.google.com/github/soerenetler/spacy_target_answer/blob/master/setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import os
os.system("git clone https://github.com/soerenetler/squad-question-generation.git")
os.system("git clone https://github.com/soerenetler/spacy_target_answer.git")
os.system("git clone https://github.com/soerenetler/crf_target_answer.git")

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!(cd spacy_target_answer; git pull)
!(cd squad-question-generation/; git pull)

Already up to date.
Already up to date.


In [17]:
!cp crf_target_answer/Custom.py Custom.py

In [3]:
!nvidia-smi

Fri Apr 16 17:57:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!pip install --quiet -U pip setuptools wheel

     |████████████████████████████████| 1.5MB 7.2MB/s 
     |████████████████████████████████| 788kB 27.8MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


In [5]:
!pip install --quiet -U spacy[cuda110,transformers,lookups]

     |████████████████████████████████| 12.8 MB 7.3 MB/s 
     |████████████████████████████████| 456 kB 73.7 MB/s 
     |████████████████████████████████| 1.1 MB 71.9 MB/s 
     |████████████████████████████████| 9.1 MB 46.4 MB/s 
     |████████████████████████████████| 165.3 MB 44 kB/s 
     |████████████████████████████████| 93.4 MB 33 kB/s 
     |████████████████████████████████| 113 kB 78.5 MB/s 
     |████████████████████████████████| 66 kB 4.3 MB/s 
     |████████████████████████████████| 998 kB 53.0 MB/s 
     |████████████████████████████████| 1.8 MB 57.2 MB/s 
     |████████████████████████████████| 2.9 MB 55.6 MB/s 
     |████████████████████████████████| 862 kB 59.1 MB/s 


In [6]:
!python -m spacy download en_core_web_sm --quiet
!python -m spacy download en_core_web_trf --quiet
!python -m spacy download en_core_web_lg --quiet

2021-04-16 17:58:09.878713: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 13.7 MB 8.5 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
2021-04-16 17:58:17.670577: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 459.7 MB 12 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')
2021-04-16 17:58:56.010110: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 778.8 MB 18 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [7]:
!spacy validate

2021-04-16 17:59:57.723179: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.0.5) =================
ℹ spaCy installation: /usr/local/lib/python3.7/dist-packages/spacy

NAME              SPACY            VERSION                            
en_core_web_sm    >=3.0.0,<3.1.0   3.0.0   ✔
en_core_web_trf   >=3.0.0,<3.1.0   3.0.0   ✔
en_core_web_lg    >=3.0.0,<3.1.0   3.0.0   ✔



In [ ]:
# Train NER
!python -m spacy project run corpus spacy_target_answer

In [ ]:
!python -m spacy init fill-config ./spacy_target_answer/configs/base_config.cfg ./spacy_target_answer/configs/config.cfg
!python -m spacy project run train spacy_target_answer

In [ ]:
!python -m spacy project run evaluate spacy_target_answer

In [12]:
import spacy
nlp = spacy.load('/content/gdrive/MyDrive/mt-qg-data/00_models/spacy_target_answer/squad/training/model-best')
doc = nlp(u"There are no major roads through Golm, only state road (Landesstraße) L 902 passes through the northern parts of the village area and offers the shortest connection to federal highway (Bundesstraße) 273.  The latter can also be reached following a road extending east from Golm past Sanssouci Park to the Jägervorstadt quarter of Potsdam. The railway between Wildpark and Nauen was opened in 1902 as part of the single-tracked bypass railway (Umgehungsbahn) intended to connect the radial railway lines originating from Berlin. A station was established in Golm. In the 1950s this line became part of the Berlin outer ring and was double-tracked. Golm is now a passenger station for regional trains to Potsdam, Wustermark, Hennigsdorf, and Berlin Schönefeld Airport. Golm has also bus connections to Potsdam and Neu Töplitz and is included in fare zone C (Tarifbereich C)[2] of Berlin's public transport system and in fare zone B of Potsdam's public transport system.")
spacy.displacy.render(doc, style='ent', jupyter =True)

In [ ]:
pip install scikit-learn sklearn_crfsuite -U

In [1]:
import pandas as pd
import spacy
from spacy.tokens import Doc
from tqdm import tqdm
import ast
from Custom import *

spacy.require_gpu()

df = pd.read_csv("/content/gdrive/MyDrive/mt-qg-data/01_data/preprocessedData/squad/paragraph/test.csv")
df["askable_tokens"] = [ast.literal_eval(t) for t in df["askable_tokens"]]
df["paragraph_tokens"] = [ast.literal_eval(t) for t in df["paragraph_tokens"]]

print(df.columns)

nlp = spacy.load('/content/gdrive/MyDrive/mt-qg-data/00_models/spacy_target_answer/squad/training/model-best')
def custom_tokenizer(nlp):
    return lambda text : Doc(nlp.vocab, words=text)

nlp.tokenizer = custom_tokenizer(nlp)


pred_y = []
true_y = []

for doc, y in tqdm(zip(nlp.pipe(df["paragraph_tokens"]), df["askable_tokens"])):
  labels = []
  for token in doc:
    if token.ent_iob_ =="O":
      labels.append("O")
    else:
      labels.append(token.ent_iob_ + "-" + token.ent_type_)
  pred_y.append(labels)
  true_y.append(y) 

Index(['Unnamed: 0', 'text_title', 'paragraph_text', 'paragraph_tokens',
       'askable_tokens'],
      dtype='object')


1204it [00:18, 65.26it/s]


In [2]:
from sklearn_crfsuite import metrics
metrics.flat_classification_report(io2iob(true_y), pred_y, digits=5)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:72: FutureWarning: Pass labels=None as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


'              precision    recall  f1-score   support\n\n    B-Answer    0.43621   0.25735   0.32372      8024\n    I-Answer    0.51863   0.15178   0.23483     27507\n           O    0.82609   0.95899   0.88759    141385\n\n    accuracy                        0.80166    176916\n   macro avg    0.59364   0.45604   0.48205    176916\nweighted avg    0.76060   0.80166   0.76053    176916\n'